In [74]:
from __future__ import division
import numpy as np
from scipy.stats import bernoulli, binom, norm
import matplotlib.pyplot as plt

mua_true=4 # we are trying to estimate this from the data
mub_true=7 # we are trying to estimate this from the data
fa=bernoulli(mua_true,1) # distribution for group A
fb=binom(mub_true,1) # distribution for group B
fz=bernoulli(0.5) # each group equally likely 
M=10
eps = 0.00001
def sample(n=10):
    'simulate picking from each group n times'
    tmp=fz.rvs(n) # choose n of the coins, A or B
    return tmp*(fb.rvs(n))+(1-tmp)*fa.rvs(n) # flip it n times

xs =  np.array([7, 7, 1, 3, 2, 1, 5, 1, 4, 0, 2, 3, 3, 1, 2, 2, 3, 1, 4, 0, 8, 3, 5, 3, 0, 0, 7, 1, 1, 3, 1, 3, 2, 4, 1, 6, 2, 2, 4, 1, 3, 1, 1, 2, 7, 3, 3, 2, 2, 2])

#a quick look at the density functions of each group and a histogram of the samples

import sympy
from sympy.abc import x, z
from sympy import stats


def ez(x, M, mu_a,mu_b, pa): # expected value of hidden variable
    pb = 1 - pa
    ev_a = binom.pmf(x, M, mu_a)*pa/(binom.pmf(x, M, mu_a)*pa+ binom.pmf(x,M, mu_b)*pb)
    ev_b = binom.pmf(x, M, mu_b)*pb/(binom.pmf(x,M,  mu_a)*pa+ binom.pmf(x, M, mu_b)*pb)
    return ev_a, ev_b




Lf=sympy.lambdify((x,mu_a,mu_b), sympy.log(abs(L)),'numpy') # convert to numpy function from sympy

def mz(x,v1, v2):
    pa_n = sum(v1) / len(x)
    pb_n = 1 - pa_n
    mu_a_n = sum(x * v1) / (10 * sum(v1))
    mu_b_n = sum(x * v2) / (10* sum(v2))

    return pa_n, pb_n, mu_a_n, mu_b_n

niter=10 #


def algorithm (x, mu_a=None, mu_b=None, pa=None):
    mu_a = mu_a or np.random.random()
    mu_b = mu_b or np.random.random()
    
    pa = pa or np.random.random()
    pb = 1 - pa
    #out=[];lout=[] # containers for outputs
    print("pa = %0.5f,  pb = %0.5f,  mu_a = %0.5f,  mu_b = %0.5f\n" % (pa, pb, mu_a, mu_b))
    print("   pa        pb       mu_a       mu_b")
    while True:
        tau_a, tau_b=ez(x,M,mu_a,mu_b, pa) # expected value of z-variable
       # lout.append( sum(Lf(x,mu_a_n,mu_b_n))) # track incomplete likelihood value (should be monotone)
      #  out.append((mu_a_n,mu_b_n))          # keep track of (pa,pb) steps
        pa_n, pb_n, mu_a_n, mu_b_n = mz(x,tau_a, tau_b)    # new maximum  likelihood estimate of pa, pb
        print(" %0.5f   %0.5f   %0.5f   %0.5f" % (pa_n, pb_n, mu_a_n, mu_b_n))
        if np.sqrt((mu_a - mu_a_n)**2 + (mu_b - mu_b_n)**2 + 2 * (pa - pa_n)**2) < eps:
            break
        mu_a, mu_b = mu_a_n, mu_b_n
        pa=pa_n
        

In [75]:
        
algorithm(xs)  


pa = 0.77446,  pb = 0.22554,  mu_a = 0.75685,  mu_b = 0.97258

   pa        pb       mu_a       mu_b
 0.99927   0.00073   0.26963   0.77217
 0.99229   0.00771   0.26618   0.76178
 0.95951   0.04049   0.25037   0.73502
 0.90989   0.09011   0.22661   0.70812
 0.88166   0.11834   0.21429   0.68504
 0.86807   0.13193   0.20948   0.66823
 0.86050   0.13950   0.20716   0.65762
 0.85590   0.14410   0.20586   0.65099
 0.85300   0.14700   0.20507   0.64679
 0.85113   0.14887   0.20457   0.64408
 0.84990   0.15010   0.20425   0.64232
 0.84910   0.15090   0.20404   0.64116
 0.84856   0.15144   0.20390   0.64039
 0.84821   0.15179   0.20381   0.63988
 0.84798   0.15202   0.20375   0.63954
 0.84782   0.15218   0.20371   0.63932
 0.84772   0.15228   0.20368   0.63917
 0.84765   0.15235   0.20366   0.63907
 0.84760   0.15240   0.20365   0.63900
 0.84757   0.15243   0.20364   0.63896
 0.84755   0.15245   0.20364   0.63893
 0.84753   0.15247   0.20364   0.63891
 0.84753   0.15247   0.20363   0.63890
 0